<a href="https://colab.research.google.com/github/Mayur619/NMT/blob/master/nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving deu.txt to deu.txt
User uploaded file "deu.txt" with length 12816559 bytes


In [0]:
!curl https://nlp.stanford.edu/data/glove.6B.zip --output glove.zip
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  822M  100  822M    0     0  24.5M      0  0:00:33  0:00:33 --:--:-- 31.3M


In [0]:
!unzip glove.zip

Archive:  glove.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,Dense,Input,Embedding
from keras.models import Model
import numpy as np

EPOCHS=100
BATCH_SIZE=64
MAX_VOCAB_SIZE=20000
NUM_SAMPLES=10000
EMBEDDING_DIM=100
LATENT_DIM=256
MAX_SEQ_LEN=100

t=0
input_texts=[]
target_texts=[]
target_texts_input=[]
for line in open('deu.txt','r'):
    if t>NUM_SAMPLES:
        break
    if '\t' not in line:
        continue
    input_text,translation=line.rstrip().split('\t')
    target_text=translation+' <eos>'
    target_text_input='<sos> '+translation
    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_input.append(target_text_input)
    t+=1

tokenizer_inputs=Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences=tokenizer_inputs.texts_to_sequences(input_texts)

tokenizer_outputs=Tokenizer(num_words=MAX_VOCAB_SIZE,filters='')
tokenizer_outputs.fit_on_texts(target_texts+target_texts_input)
targets_sequences=tokenizer_outputs.texts_to_sequences(target_texts)
targets_text_inputs_sequences=tokenizer_outputs.texts_to_sequences(target_texts_input)

word2idx_inputs=tokenizer_inputs.word_index
print('Found %d unique input tokens'%(len(word2idx_inputs)))
word2idx_outputs=tokenizer_outputs.word_index
print('Found %d unique output tokens'%(len(word2idx_outputs)))

max_len_input=max(len(s) for s in input_sequences)
max_len_targets=max(len(s) for s in targets_sequences)

num_words_outputs=len(word2idx_outputs)+1
encoder_inputs=pad_sequences(input_sequences,maxlen=max_len_input,padding='post')
decoder_inputs=pad_sequences(targets_text_inputs_sequences,maxlen=max_len_targets,padding='post')
decoder_targets=pad_sequences(targets_sequences,maxlen=max_len_targets,padding='post')

print('Loading word vectors....')
word2vec={}
for line in open('glove.6B.100d.txt'):
    line=line.split()
    word=line[0]
    vec=np.asarray(line[1:],dtype=np.float32)
    word2vec[word]=vec

print('Applying pre-trained glove....')
num_words=min(MAX_VOCAB_SIZE,len(word2idx_inputs)+1)
embedding_matrix=np.zeros((num_words,EMBEDDING_DIM))
for word,idx in word2idx_inputs.items():
    if idx<MAX_VOCAB_SIZE:
        vector=word2vec.get(word)
        if vector is not None:
            embedding_matrix[idx]=vector

embedding_layer=Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_len_input)
decoder_targets_one_hot=np.zeros((len(input_texts),max_len_targets,num_words_outputs))
for i,target in enumerate(decoder_targets):
    for t,word in enumerate(target):
        decoder_targets_one_hot[i,t,word]=1

encoder_input_placeholder=Input(shape=(max_len_input,))
x=embedding_layer(encoder_input_placeholder)
encoder=LSTM(LATENT_DIM,return_state=True)
encoder_out,h,c=encoder(x)
encoder_state=[h,c]

decoder_input_placeholder=Input(shape=(max_len_targets,))
decoder_embedding=Embedding(num_words_outputs,LATENT_DIM)
decoder_input_x=decoder_embedding(decoder_input_placeholder)
decoder=LSTM(LATENT_DIM,return_sequences=True,return_state=True)
decoder_out,_,_=decoder(decoder_input_x,initial_state=encoder_state)
decoder_dense=Dense(num_words_outputs,activation='softmax')
decoder_output=decoder_dense(decoder_out)
seq2seq=Model([encoder_input_placeholder,decoder_input_placeholder],decoder_output)

seq2seq.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
seq2seq.fit([encoder_inputs,decoder_inputs],decoder_targets_one_hot,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2)
seq2seq.save('s2sg.h5')

encoder_model=Model(encoder_input_placeholder,encoder_state)
decoder_input_h=Input(shape=(LATENT_DIM,))
decoder_input_c=Input(shape=(LATENT_DIM,))
decoder_states_inputs=[decoder_input_h,decoder_input_c]
decoder_input_single=Input(shape=(1,))
decoder_input_single_x=decoder_embedding(decoder_input_single)
decoder_outputs,h,c=decoder(decoder_input_single_x,initial_state=decoder_states_inputs)
decoder_states=[h,c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model([decoder_input_single]+decoder_states_inputs,[decoder_outputs]+decoder_states)

idx2word={v:k for k,v in word2idx_inputs.items()}
idx2trans={v:k for k,v in word2idx_outputs.items()}

def translate(input_seq):
    states_value=encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1))
    target_seq[0,0]=word2idx_outputs['<sos>']
    eos=word2idx_outputs['<eos>']
    output_sentence=[]
    for _ in range(max_len_targets):
        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
        idx=np.argmax(output_tokens[0,0,:])
        if eos==idx:
            break
        if idx>0:
            output_sentence.append(idx2trans[idx])
        target_seq[0,0]=idx
        states_value=[h,c]
    return ' '.join(output_sentence)

while True:
  input_=input('Input:')
  translation = translate(pad_sequences(tokenizer_inputs.texts_to_sequences([input_]),maxlen=max_len_input,padding='post'))
  print('Input:', input_)
  print('Translation:', translation)

  ans = input("Continue? [Y/n]")
  if ans and ans.lower().startswith('n'):
      break

Using TensorFlow backend.


Found 2332 unique input tokens
Found 5207 unique output tokens
Loading word vectors....
Applying pre-trained glove....
Train on 8000 samples, validate on 2001 samples
Epoch 1/100
8000/8000 [==============================] - 14s 2ms/step - loss: 2.3176 - acc: 0.6853 - val_loss: 2.2745 - val_acc: 0.6868
Epoch 2/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.6579 - acc: 0.7534 - val_loss: 2.0551 - val_acc: 0.7172
Epoch 3/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.4563 - acc: 0.7783 - val_loss: 1.9302 - val_acc: 0.7322
Epoch 4/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.3362 - acc: 0.7917 - val_loss: 1.8746 - val_acc: 0.7424
Epoch 5/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.2378 - acc: 0.8034 - val_loss: 1.8065 - val_acc: 0.7517
Epoch 6/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.1517 - acc: 0.8131 - val_loss: 1.7781 - val_acc: 0.7588
Epoch 7/100
8000/

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Input:Hi.
Input: Hi.
Translation: hallo!
Continue? [Y/n]y
Input:Bye.
Input: Bye.
Translation: stürmte karte.
Continue? [Y/n]y
Input:Hug me.
Input: Hug me.
Translation: drück mich!
Continue? [Y/n]
Input:you won.
Input: you won.
Translation: du hast gewonnen.
Continue? [Y/n]n


In [0]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Input,LSTM,Embedding
from keras.models import Model
import numpy as np

EPOCHS=100
BATCH_SIZE=128
MAX_VOCAB_SIZE=20000
NUM_SAMPLES=10000
EMBEDDING_DIM=100
LATENT_DIM=256
MAX_SEQ_LEN=100

t=0
input_texts=[]
target_texts=[]
target_texts_input=[]
for line in open('deu.txt','r'):
    if t>NUM_SAMPLES:
        break
    if '\t' not in line:
        continue
    input_text,translation=line.rstrip().split('\t')
    target_text=translation+' <eos>'
    target_text_input='<sos> '+translation
    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_input.append(target_text_input)
    t+=1

tokenizer_inputs=Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences=tokenizer_inputs.texts_to_sequences(input_texts)

tokenizer_outputs=Tokenizer(num_words=MAX_VOCAB_SIZE,filters='')
tokenizer_outputs.fit_on_texts(target_texts+target_texts_input)
targets_sequences=tokenizer_outputs.texts_to_sequences(target_texts)
targets_text_inputs_sequences=tokenizer_outputs.texts_to_sequences(target_texts_input)

word2idx_inputs=tokenizer_inputs.word_index
print('Found %d unique input tokens'%(len(word2idx_inputs)))
word2idx_outputs=tokenizer_outputs.word_index
print('Found %d unique output tokens'%(len(word2idx_outputs)))

max_len_input=max(len(s) for s in input_sequences)
max_len_targets=max(len(s) for s in targets_sequences)

num_words_outputs=len(word2idx_outputs)+1
encoder_inputs=pad_sequences(input_sequences,maxlen=max_len_input,padding='post')
decoder_inputs=pad_sequences(targets_text_inputs_sequences,maxlen=max_len_targets,padding='post')
decoder_targets=pad_sequences(targets_sequences,maxlen=max_len_targets,padding='post')

print('Loading word vectors....')
word2vec={}
for line in open('glove.6B.100d.txt'):
    line=line.split()
    word=line[0]
    vec=np.asarray(line[1:],dtype=np.float32)
    word2vec[word]=vec

print('Applying pre-trained glove....')
num_words=min(MAX_VOCAB_SIZE,len(word2idx_inputs)+1)
embedding_matrix=np.zeros((num_words,EMBEDDING_DIM))
for word,idx in word2idx_inputs.items():
    if idx<MAX_VOCAB_SIZE:
        vector=word2vec.get(word)
        if vector is not None:
            embedding_matrix[idx]=vector

word2vec={}
embedding_layer=Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_len_input)
decoder_targets_one_hot=np.zeros((len(input_texts),max_len_targets,num_words_outputs))
for i,target in enumerate(decoder_targets):
    for t,word in enumerate(target):
        decoder_targets_one_hot[i,t,word]=1

encoder_input_placeholder=Input(shape=(max_len_input,))
x=embedding_layer(encoder_input_placeholder)
encoder_lstm_1=LSTM(LATENT_DIM,return_sequences=True)(x)
encoder_lstm_2=LSTM(LATENT_DIM,return_sequences=True,return_state=True)
encoder_out,h,c=encoder_lstm_2(encoder_lstm_1)
encoder_state=[h,c]

decoder_input_placeholder=Input(shape=(max_len_targets,))
decoder_embedding=Embedding(num_words_outputs,LATENT_DIM)
decoder_input_x=decoder_embedding(decoder_input_placeholder)
decoder_lstm_1=LSTM(LATENT_DIM,return_sequences=True)(decoder_input_x)
decoder=LSTM(LATENT_DIM,return_state=True,return_sequences=True)
decoder_out,_,_=decoder(decoder_lstm_1,initial_state=encoder_state)
decoder_dense=Dense(num_words_outputs,activation='softmax')
decoder_output=decoder_dense(decoder_out)
seq2seq=Model([encoder_input_placeholder,decoder_input_placeholder],decoder_output)
#seq2seq.load_weights('s2sg_weights.h5')
seq2seq.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
seq2seq.fit([encoder_inputs,decoder_inputs],decoder_targets_one_hot,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2)
#seq2seq.save('s2sg.h5')

encoder_model=Model(encoder_input_placeholder,encoder_state)
decoder_input_h=Input(shape=(LATENT_DIM,))
decoder_input_c=Input(shape=(LATENT_DIM,))
decoder_states_inputs=[decoder_input_h,decoder_input_c]
decoder_input_single=Input(shape=(1,))
decoder_input_single_x=decoder_embedding(decoder_input_single)
decoder_outputs,h,c=decoder(decoder_input_single_x,initial_state=decoder_states_inputs)
decoder_states=[h,c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model([decoder_input_single]+decoder_states_inputs,[decoder_outputs]+decoder_states)

idx2word={v:k for k,v in word2idx_inputs.items()}
idx2trans={v:k for k,v in word2idx_outputs.items()}

def translate(input_seq):
    states_value=encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1))
    target_seq[0,0]=word2idx_outputs['<sos>']
    eos=word2idx_outputs['<eos>']
    output_sentence=[]
    for _ in range(max_len_targets):
        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
        idx=np.argmax(output_tokens[0,0,:])
        if eos==idx:
            break
        if idx>0:
            output_sentence.append(idx2trans[idx])
        target_seq[0,0]=idx
        states_value=[h,c]
    return ' '.join(output_sentence)

while True:
  input_=input('Input:')
  translation = translate(pad_sequences(tokenizer_inputs.texts_to_sequences([input_]),maxlen=max_len_input,padding='post'))
  print('Input:', input_)
  print('Translation:', translation)

  ans = input("Continue? [Y/n]")
  if ans and ans.lower().startswith('n'):
      break

Using TensorFlow backend.


Found 2332 unique input tokens
Found 5207 unique output tokens
Loading word vectors....
Applying pre-trained glove....
Train on 8000 samples, validate on 2001 samples
Epoch 1/100
8000/8000 [==============================] - 16s 2ms/step - loss: 2.5984 - acc: 0.6459 - val_loss: 2.4635 - val_acc: 0.6684
Epoch 2/100
8000/8000 [==============================] - 13s 2ms/step - loss: 1.8939 - acc: 0.7237 - val_loss: 2.2953 - val_acc: 0.6887
Epoch 3/100
8000/8000 [==============================] - 13s 2ms/step - loss: 1.6826 - acc: 0.7432 - val_loss: 2.1851 - val_acc: 0.6913
Epoch 4/100
8000/8000 [==============================] - 13s 2ms/step - loss: 1.5297 - acc: 0.7574 - val_loss: 2.1032 - val_acc: 0.7091
Epoch 5/100
8000/8000 [==============================] - 13s 2ms/step - loss: 1.4142 - acc: 0.7728 - val_loss: 2.0599 - val_acc: 0.7230
Epoch 6/100
8000/8000 [==============================] - 13s 2ms/step - loss: 1.3238 - acc: 0.7842 - val_loss: 2.0499 - val_acc: 0.7146
Epoch 7/100
8000/

In [0]:
seq2seq.save_weights('s2sg_weights.h5')

In [0]:
from google.colab import files

files.download('s2sg_weights.h5')